In [19]:
import psycopg2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
import os

In [4]:
import mlflow
import mlflow.sklearn

In [6]:
mlflow.set_tracking_uri("http://mlflow:5000")

In [8]:
experiments = {exp.name: exp.experiment_id for exp in mlflow.search_experiments()}

if "random_forest_model" in experiments:
    mlflow.set_experiment(experiment_id=experiments["random_forest_model"])
else:
    mlflow.create_experiment("random_forest_model")
    mlflow.set_experiment("random_forest_model")

In [22]:
# Conexión con PostgreSQL
conn = psycopg2.connect(
    dbname="mlops_db",
    user="user",
    password="password",
    host="postgres",
    port="5432"
)
cur = conn.cursor()

In [24]:
create_table_query = """
CREATE TABLE IF NOT EXISTS resultados_ml (
    id SERIAL PRIMARY KEY,
    experimento VARCHAR(100),
    modelo VARCHAR(100),
    mse FLOAT
);
"""

In [25]:
cur.execute(create_table_query)
conn.commit()

In [9]:
_data_root = './data/'

In [10]:
_data_filepath = os.path.join(_data_root, 'train_preprocessed.csv')

In [11]:
file_path = "./data/train_preprocessed.csv"
df = pd.read_csv(file_path)

In [12]:
# Separar características y etiquetas
X = df.drop(columns=["Cover_Type"])
y = df["Cover_Type"]

In [13]:
X = pd.get_dummies(X, drop_first=True)
X

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_C7755,Soil_Type_C7756,Soil_Type_C7757,Soil_Type_C7790,Soil_Type_C8703,Soil_Type_C8707,Soil_Type_C8708,Soil_Type_C8771,Soil_Type_C8772,Soil_Type_C8776
0,3305,35,13,134,25,5811,218,211,127,659,...,False,False,False,False,False,False,False,True,False,False
1,3295,332,10,934,160,924,196,227,170,2519,...,False,True,False,False,False,False,False,False,False,False
2,3320,339,12,711,106,979,195,223,167,2531,...,False,False,False,False,False,False,False,False,False,False
3,3321,16,14,785,193,3221,207,209,137,787,...,False,False,False,False,False,False,False,False,False,False
4,2959,233,17,190,85,1519,189,253,198,999,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92957,2324,26,33,124,67,1473,185,150,81,1034,...,False,False,False,False,False,False,False,False,False,False
92958,2384,144,6,95,10,604,230,239,143,934,...,False,False,False,False,False,False,False,False,False,False
92959,3287,96,6,67,4,4496,231,231,133,3684,...,False,False,False,False,False,False,False,False,False,False
92960,2960,344,8,30,3,3193,205,228,162,1176,...,False,False,False,False,False,False,False,False,False,False


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Definir hiperparámetros a probar
n_estimators_list = [10, 50, 100, 200]
max_depth_list = [None, 5, 10, 20]

In [26]:
for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        with mlflow.start_run():  # Iniciar un experimento en MLflow
            # Crear y entrenar modelo
            model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)

            insert_query = """
            INSERT INTO resultados_ml (experimento, modelo, mse)
            VALUES (%s, %s, %s);
            """
            
            # Datos a insertar
            nuevo_resultado = (n_estimators, "RandomForestClassifier", acc)
            
            cur.execute(insert_query, nuevo_resultado)
            conn.commit()

            # Registrar parámetros y métricas en MLflow
            mlflow.log_param("n_estimators", n_estimators)
            mlflow.log_param("max_depth", max_depth)
            mlflow.log_metric("accuracy", acc)
            mlflow.sklearn.log_model(model, "random_forest_model")

            print(f'n_estimators={n_estimators}, max_depth={max_depth}, Accuracy={acc:.5f}')


2025/03/19 04:23:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


n_estimators=10, max_depth=None, Accuracy=0.87796
🏃 View run learned-fawn-50 at: http://mlflow:5000/#/experiments/1/runs/ddadb7681a7e43e39200ee7d900c0689
🧪 View experiment at: http://mlflow:5000/#/experiments/1


2025/03/19 04:23:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


n_estimators=10, max_depth=5, Accuracy=0.67230
🏃 View run popular-auk-378 at: http://mlflow:5000/#/experiments/1/runs/c3ee257191bb4e1790112076ebdc49f6
🧪 View experiment at: http://mlflow:5000/#/experiments/1


2025/03/19 04:23:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

n_estimators=200, max_depth=5, Accuracy=0.66955
🏃 View run sneaky-horse-842 at: http://mlflow:5000/#/experiments/1/runs/c6fe02ca20c24d94b43b435b14271db6
🧪 View experiment at: http://mlflow:5000/#/experiments/1


2025/03/19 04:25:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


n_estimators=200, max_depth=10, Accuracy=0.75464
🏃 View run redolent-zebra-239 at: http://mlflow:5000/#/experiments/1/runs/ba904055f5884b5e9550323ebc28d5ad
🧪 View experiment at: http://mlflow:5000/#/experiments/1


2025/03/19 04:25:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


n_estimators=200, max_depth=20, Accuracy=0.85952
🏃 View run incongruous-stag-725 at: http://mlflow:5000/#/experiments/1/runs/44379cda251c4372a670750a0b9e5542
🧪 View experiment at: http://mlflow:5000/#/experiments/1


In [17]:
# Entrenamiento del modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [18]:
# Predicción y evaluación
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Precisión del modelo: {accuracy:.4f}")
print(f"Precisión: {precision:.4f}")
print(f"Sensibilidad (Recall): {recall:.4f}")
print(f"Puntaje F1: {f1:.4f}")

Precisión del modelo: 0.9024
Precisión: 0.9028
Sensibilidad (Recall): 0.9024
Puntaje F1: 0.9012


In [27]:
select_query = "SELECT * FROM resultados_ml;"

cur.execute(select_query)
resultados = cur.fetchall()

for row in resultados:
    print(row)

cur.close()
conn.close()

(1, '10', 'RandomForestClassifier', 0.8779648254719518)
(2, '10', 'RandomForestClassifier', 0.6722960253858979)
(3, '10', 'RandomForestClassifier', 0.7522723605658044)
(4, '10', 'RandomForestClassifier', 0.8534932501479051)
(5, '50', 'RandomForestClassifier', 0.8993169472382079)
(6, '50', 'RandomForestClassifier', 0.6724035927499596)
(7, '50', 'RandomForestClassifier', 0.7487764212337976)
(8, '50', 'RandomForestClassifier', 0.8575808099822514)
(9, '100', 'RandomForestClassifier', 0.9023826171139676)
(10, '100', 'RandomForestClassifier', 0.6696606249663852)
(11, '100', 'RandomForestClassifier', 0.7525950626579896)
(12, '100', 'RandomForestClassifier', 0.8608616145861345)
(13, '200', 'RandomForestClassifier', 0.9013069434733502)
(14, '200', 'RandomForestClassifier', 0.6695530576023234)
(15, '200', 'RandomForestClassifier', 0.7546388425751627)
(16, '200', 'RandomForestClassifier', 0.8595170225353628)
